In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
import pandas as pd
import sys
from os import path
import utils
from data_flow import DataFlow, gs_transfer

base_dir = path.join('../../', 'dataset/AgriculturalDisease')

tr_dir = path.join(base_dir, 'trainingset')
v_dir = path.join(base_dir, 'validationset')
t_dir = path.join(base_dir, 'testA')

sc cls: 10, ds cls: 20, sr cls: 4


## run model

In [2]:
from gs_model import GS_Model

model = GS_Model(256)

initiating image keys
building placeholder
buliding model
--- building sr cls block
--- building sp cls block
--- building ds cls block
--- building full cls block
building losses
building optimizer & train_ops
trainable: 1508621
global: 4527669.0

model_version: 1.0.5.1021


In [3]:
%%time
# model.lr = 1e-5
# model.lr_decay = 0.5
# model.batch_size = 512
model.train(10, go_on=False)

current losses: [1.1706179  1.9232918  2.6570454  0.59677374], last: 9.8, saving model
valid loss: ['1.170618', '1.923292', '2.657045', '0.596774']
current losses: [0.92773324 1.5405495  2.3546746  0.53635585], last: 0.5967737436294556, saving model
valid loss: ['0.927733', '1.54055', '2.354674', '0.536356']
current losses: [0.8262634  1.4061394  2.1961606  0.51689184], last: 0.5363558530807495, saving model
valid loss: ['0.826263', '1.406139', '2.19616', '0.516892']
current losses: [0.7857272 1.4021276 2.18481   0.4594411], last: 0.5168918371200562, saving model
valid loss: ['0.785727', '1.402128', '2.18481', '0.459441']
current losses: [0.7736488 1.3953121 2.1747236 0.4581639], last: 0.45944109559059143, saving model
valid loss: ['0.773649', '1.395312', '2.174724', '0.458164']
valid loss: ['0.768215', '1.387398', '2.164172', '0.460385']02']
valid loss: ['0.766605', '1.388841', '2.163334', '0.458868']06']
valid loss: ['0.766269', '1.38709', '2.162578', '0.460825']93']]


KeyboardInterrupt: 

## graph test

In [7]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('../../logs/agr-ds', graph=sess.graph)

## model test

In [4]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    model_path = tf.train.latest_checkpoint('../../ckpt/agr-ds')
    saver.restore(sess, model_path)
    
    feeder = model.get_feeder('validation', batch_size=1024)
    pred, lb = [], []
    for img_data, sr, sp, ds, full_lb in feeder:
        feed_dict = {
            model.pl_in_data: img_data,
            model.pl_training: False
        }
        _pred = sess.run(tf.nn.softmax(model.full_logits), feed_dict=feed_dict)
        
        pred.append(_pred)
        lb.append(full_lb)

pred = np.vstack(pred)
lb = np.vstack(lb)

p = np.argmax(pred, axis=1)
l = np.argmax(lb, axis=1)

sum(p==l)/len(p)

INFO:tensorflow:Restoring parameters from ../../ckpt/agr-ds\model.ckpt-4


0.8108326596604689

## transfer

In [1]:
from data_flow import gs_transfer

sc cls: 10, ds cls: 20, sr cls: 4


In [2]:
gs_transfer()

gsm loaded
dataflow loaded
begin transfer
this epoch has total 31 batches
tr transfer finished
this epoch has total 4 batches
v transfer finished


# Prediction

## refine test img

In [1]:
import data_flow
import h5py

sc cls: 10, ds cls: 20, sr cls: 4


In [2]:
data_flow.extract_test_features()

total: 5


In [3]:
test_h5_path = 'f:/dataset/AgriculturalDisease/test.h5'
with h5py.File(test_h5_path, 'r') as f:
    print(len(f.keys()))

4959


#### -----------------

In [11]:
data = model.pred_test()

INFO:tensorflow:Restoring parameters from ../../ckpt/agr-ds/model.ckpt-6
total:  5
dealing with batch 1/5
dealing with batch 2/5
dealing with batch 3/5
dealing with batch 4/5
dealing with batch 5/5


In [12]:
for d in data:
    d['disease_class'] = str(d['disease_class'])

In [14]:
import json
with open('pred.json', 'w') as f:
    json.dump(data, f)

## test

In [4]:
import tensorflow as tf
import numpy as np
import h5py
import json
import utils
import json
from gs_model import GS_Model


In [5]:
with h5py.File('f:/dataset/AgriculturalDisease/test.h5', 'r') as f:
    keys = [k for k in f.keys()]
    data = f[keys[0]].value

In [6]:
data.shape

(8, 8, 1920)

In [7]:
model = GS_Model()

initiating image keys
key validation ------------
k0 ecc8d9d8
k1 eec980f4
k2 04e2d58c
key validation ------------
tr: 29388, v: 3000 test: 3000
building placeholder
buliding model
--- building sr cls block
--- building sp cls block
--- building ds cls block
--- building full cls block
building losses
building optimizer & train_ops
trainable: 433525
global: 1301877.0

model_version: 1.0.5.1021


In [12]:
json_data = model.pred_test()

INFO:tensorflow:Restoring parameters from ../../ckpt/agr-ds/model.ckpt-9
total:  5
dealing with batch 1/5
dealing with batch 2/5
dealing with batch 3/5
dealing with batch 4/5
dealing with batch 5/5


In [39]:
import json
for pred in json_data:
    pred['disease_class'] = str(pred['disease_class'])
    
with open('pred.json', 'w') as f:
    json.dump(json_data, f)

In [40]:
with open('f:/dataset/AgriculturalDisease/trainingset/labels.json') as f:
    tr_json = json.load(f)
with open('f:/dataset/AgriculturalDisease/validationset/labels.json') as f:
    v_json = json.load(f)

print(len(tr_json), len(v_json))

32739 4982


In [41]:
json_dict = {j['image_id']:j['disease_class'] for j in json_data}

In [42]:
all_json = tr_json + v_json
len(all_json)

37721

In [43]:
all_id = [x['image_id'] for x in all_json]

In [44]:
X = []
for pred in json_data:
    img_id = pred['image_id'].split('__')[0]
    img_id = img_id.split(' - 副本')[0]
    if img_id in all_id:
        X.append(pred['image_id'].split('.jpg')[0])
len(X)

236

In [45]:
tr_dict = {k['image_id']: k['disease_class'] for k in tr_json}
v_dict = {k['image_id']: k['disease_class'] for k in v_json}
len(tr_dict), len(v_dict), len(all_json)

(32739, 4982, 37721)

In [46]:
def findit(xx, jj, val):
    for j in jj:
        idx = j['image_id']
        data = j['disease_class']
        
        if xx in idx and str(data) != str(val):
            return xx, data, val
            

In [47]:
rslt = {}
for idx in X:
    val = json_dict[idx+'.jpg']
    tr_fk = findit(idx, tr_json, val)
    v_fk = findit(idx, v_json, val)
    if tr_fk != None or v_fk != None:
        rslt[idx] = [tr_fk, v_fk]

len(rslt)

89

In [48]:
for k, v in rslt.items():
    v = (v[0] or v[1])[1]
    json_dict[k+'.jpg'] = v

In [49]:
result = [{ 'image_id': k, 'disease_class': v } for k, v in json_dict.items()]

In [50]:
x2 = []
for r in result:
    for s in json_data:
        if (r['image_id'] == s['image_id']) and (r['disease_class'] != s['disease_class']):
            x2.append(r['image_id'])

len(x2)

89

In [ ]:
for x, y in zip(json_data, result):
    if x['disease_class'] != y['disease_class']:
        print(x['image_id'], x['disease_class'], y['disease_class'])

In [59]:
for idx in x2:
    if idx.split('.jpg')[0] not in X:
        print('fk')

In [54]:
with open('result.json', 'w') as f:
    json.dump(result, f)

## backup ckpt

In [76]:
!ls -l ../../ckpt/agr-ds/

total 7985
drwxr-xr-x 1 Herdsric 197121       0 Sep 16 00:31 bak
-rw-r--r-- 1 Herdsric 197121      81 Sep 16 12:25 checkpoint
-rw-r--r-- 1 Herdsric 197121 7247964 Sep 16 12:25 model.ckpt-0.data-00000-of-00001
-rw-r--r-- 1 Herdsric 197121    3428 Sep 16 12:25 model.ckpt-0.index
-rw-r--r-- 1 Herdsric 197121  914350 Sep 16 12:25 model.ckpt-0.meta


In [79]:
!ls -l ../../ckpt/agr-ds/bak

total 7981
-rw-r--r-- 1 Herdsric 197121      81 Sep 16 00:30 checkpoint
-rw-r--r-- 1 Herdsric 197121 7247964 Sep 16 00:31 model.ckpt-0.data-00000-of-00001
-rw-r--r-- 1 Herdsric 197121    3428 Sep 16 00:31 model.ckpt-0.index
-rw-r--r-- 1 Herdsric 197121  914350 Sep 16 00:31 model.ckpt-0.meta


## statistics

In [67]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
import pandas as pd
import sys
from os import path
import utils
from data_flow import DataFlow, gs_transfer
from gs_model import GS_Model

model = GS_Model(256)

base_dir = path.join('../../', 'dataset/AgriculturalDisease')

tr_dir = path.join(base_dir, 'trainingset')
v_dir = path.join(base_dir, 'validationset')
t_dir = path.join(base_dir, 'testA')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
initiating image keys
building placeholder
buliding model
--- building sr cls block
--- building sp cls block
--- building ds cls block
--- building full cls block
building losses
building optimizer & train_ops
trainable: 603919
global: 1811991.0

model_version: 1.0.21


In [336]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    model_path = tf.train.latest_checkpoint('../../ckpt/agr-ds/bak')
    saver.restore(sess, model_path)
    
    feeder = model.get_feeder('validation')
    pred, lb = [], []
    for img_data, sr, sp, ds, full_lb in feeder:
        feed_dict = {
            model.pl_in_data: img_data,
            model.pl_training: False
        }
        _full, _sr, _sp, _ds = sess.run([
            tf.nn.softmax(model.full_logits),
            tf.nn.softmax(model.sr_logits),
            tf.nn.softmax(model.sp_logits),
            tf.nn.softmax(model.ds_logits),
        ], feed_dict=feed_dict)
        
        if pred == []:
            pred = [_sr, _sp, _ds, _full]
            lb = [sr, sp, ds, full_lb]
        else:
            for idx, (x, y) in enumerate(zip(pred, [_sr, _sp, _ds, _full])):
                pred[idx] = np.vstack([x, y])
            for idx, (x, y) in enumerate(zip(lb, [sr, sp, ds, full_lb])):
                lb[idx] = np.vstack([x, y])
        

p = np.argmax(pred[-1], axis=1)
l = np.argmax(lb[-1], axis=1)

sum(p==l)/len(p)

sc cls: 10, ds cls: 20, sr cls: 4
INFO:tensorflow:Restoring parameters from ../../ckpt/agr-ds/bak\model.ckpt-0


0.7737856282617422

In [355]:
pred_ = [None]*4
for idx, d in enumerate(pred):
    pred_[idx] = np.argmax(d, axis=1)

In [356]:
lb_ = [None]*4
for idx, d in enumerate(lb):
    lb_[idx] = np.argmax(d, axis=1)

In [357]:
sum(pred_[3] == lb_[3])/len(pred_[3])

0.7737856282617422

In [358]:
# test utils.rectify_label
for idx, (sr, sp, ds, full) in enumerate(zip(*pred_)):
    full = utils.rectify_bad_label(sr, sp, ds, full)
    pred_[3][idx] = full

In [359]:
sum(pred_[3] == lb_[3])/len(pred_[3])

0.7370533922119631

### ---------

In [291]:
import utils

sc cls: 10, ds cls: 20, sr cls: 4


In [319]:
# full对了，但是sr, sp, ds其中之一不对
fk_data = []
for idx, key, sr, sp, ds, full, _sr, _sp, _ds, _full in zip(range(len(model.v_keys)), model.v_keys, *lb_, *pred_):
    if full == _full and (sr != _sr or sp != _sp or ds != _ds):
        fk_data.append([idx, key, sr, sp, ds, full])

In [287]:
# full错了，但是sr, sp, ds都对
fk_data = []
for idx, key, sr, sp, ds, full, _sr, _sp, _ds, _full in zip(range(len(model.v_keys)), model.v_keys, *lb_, *pred_):
    if full != _full and sr == _sr and sp == _sp and ds == _ds:
        fk_data.append([idx, key, sr, sp, ds, full])

In [332]:
# full对，且sr, sp, ds, full是bad_label
fk_data = []
for idx, key, sr, sp, ds, full, _sr, _sp, _ds, _full in zip(range(len(model.v_keys)), model.v_keys, *lb_, *pred_):
    if full != _full and utils.bad_label(_sr, _sp, _ds, _full) == 1:
        fk_data.append([idx, key, sr, sp, ds, full])

In [334]:
df[df.iloc[:, 3] == 3]

,Label name(标签名),物种,疾病,严重程度
Label id,,,,
3,Apple Frogeye Spot（苹果灰斑病）,0,2,3
16,Maize dwarf mosaic virus（玉米花叶病毒病）,2,8,3
60,Tomato Tomv（番茄花叶病毒病）,9,8,3
